Hello, let me explained this chatbot. 

Why i'm rename Simple Chatbot? because this ChatBot just only using Clasification Concept. We have list of input text conversation and output conversation. This chatbot using data history in dataset, so if this Chatbot looking sentences not in training, it cann't detection. I'm not sure this chatbot can be responses using new data from user, but if we want use like that, we can using Generate Chatbot. This my argument has I know, but if you have more knowladge, let's discussion soon. 

In [42]:
#load data
import json
import pandas as pd

with open('dataset/humanText.json') as data_file:
    data = json.load(data_file)

In [45]:
textInput = []
humanText = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        textInput.append(pattern)
        humanText.append(intent['tag'])

df = pd.DataFrame({'textInput': textInput,
                    'intents': humanText})
df.head()

,textInput,intents
0,Halo Ara,salam
1,Halo,salam
2,Halo,salam
3,Selamat Pagi,salam
4,Selamat Siang,salam


In [46]:
df.intents.value_counts()

kronologi             83
laporOrang            70
usiaPelaku            40
pekerjaan             33
salam                 28
hubungan              27
tempat                27
laporDiri             22
pendidikanKorban      20
adaKorbanLain         14
pendidikanPelaku      11
bye                   11
korbanLain             8
tidakadaKorbanLain     7
Name: intents, dtype: int64

# Pre Processing

In [48]:
# data cleansing
import string

# convert lowercase
df.textInput = df.textInput.apply(lambda x: x.lower())

# remove punctuation
exclude = set(string.punctuation)
df.textInput = df.textInput.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

### Label Encoding to labelling tag category

In [49]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y_train = le.fit_transform(df.intents)
y_train = to_categorical(y_train)

In [50]:
all_vocab = []
length = []

for idx, row in df.iterrows():
    sent = row['textInput']
    [all_vocab.append(i) for i in sent.split()]
    length.append(len(sent.split()))

In [31]:
len(all_vocab)

2538

In [32]:
max(length)

152

In [33]:
len(set(all_vocab))

828

### Vectorization to convert sentences be vector

In [35]:
from tensorflow.keras.layers import TextVectorization

max_vocab_length = 462
max_length = 152

text_vectorization = TextVectorization(max_tokens=max_vocab_length,
                                       standardize='lower_and_strip_punctuation',
                                       split='whitespace',
                                       ngrams=None,
                                       output_mode='int',
                                       output_sequence_length=max_length
                                       )

In [36]:
text_vectorization.adapt(df.textInput)
#text_vectorization.get_vocabulary()
#text_vectorization('halo ara ingin membantu')

### Embedding to create arrary from sentences

In [37]:
from tensorflow.keras.layers import Embedding
embedding = Embedding(input_dim=max_vocab_length,
                      output_dim=16,
                      embeddings_initializer="uniform",
                      input_length=max_length)
import numpy as np
res_embed = embedding(np.array([[71, 17,  7, 13,  0,  0]]))
res_embed

<tf.Tensor: shape=(1, 6, 16), dtype=float32, numpy=
array([[[ 0.04252667, -0.01084045, -0.01178681,  0.03017611,
         -0.00412071, -0.02765962,  0.03103551,  0.02681298,
         -0.04006124,  0.02020841,  0.01969254, -0.04620464,
         -0.03793932, -0.03745115,  0.00915391, -0.01976777],
        [-0.01660417, -0.00672623,  0.00442706, -0.04289648,
         -0.04893948, -0.02255445,  0.01061283,  0.0231486 ,
          0.04656494, -0.01453026, -0.01130132,  0.01887038,
         -0.00403232, -0.01606822, -0.04683238, -0.0449682 ],
        [ 0.04138238, -0.00832059, -0.00115852,  0.01247829,
          0.02694828,  0.04907492,  0.03925398,  0.03525894,
         -0.04290357, -0.04658737, -0.03474482,  0.00431371,
          0.03371746, -0.04131329,  0.0385217 , -0.04083408],
        [ 0.04507855, -0.01723065,  0.00276566, -0.00999359,
         -0.00655423,  0.04067147, -0.04068791, -0.0257197 ,
          0.03716056, -0.03221033,  0.03156299,  0.0385995 ,
          0.00235199, -0.00343

# Modelling

### LSTM

In [38]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, LSTM
inputs = Input(shape=(1,), dtype='string')
x = text_vectorization(inputs)
x = embedding(x)
x = LSTM(12)(x)
outputs = Dense(13, activation='softmax')(x)
modelLSTM = Model(inputs, outputs, name="LSTM_model")

In [39]:
# compile model
modelLSTM.compile(loss='categorical_crossentropy',
                   optimizer='adam',
                   metrics=["accuracy"])

In [41]:
modelLSTM.fit(df.textInput,
               y_train,
               epochs=150,
               verbose=1)

Epoch 1/150


ValueError: in user code:

    File "c:\Users\melli\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\melli\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\melli\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\melli\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\melli\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "c:\Users\melli\Ap

In [ ]:
modelLSTM.evaluate(df.textInput, y_train)

In [ ]:
modelLSTM.save("modelLSTM.tf")

In [ ]:
import pickle
le_filename = open("label_encoder.pickle", "wb")
pickle.dump(le, le_filename)
le_filename.close()

### Neural Network